####NAME - MUSKAN SHARMA
####BITS ID - 2025AA05662
####ML Assignment 2

import csv file and create dataframe

In [ ]:
import pandas as pd
import numpy as np
import requests
import zipfile
from io import BytesIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score,roc_auc_score,precision_score,recall_score,f1_score,matthews_corrcoef,confusion_matrix)
dataset_url = "https://archive.ics.uci.edu/static/public/222/bank+marketing.zip"
zip_file = requests.get(dataset_url)          #downloading  zip
zip_file.raise_for_status()
with zipfile.ZipFile(BytesIO(zip_file.content)) as outer_zip_file:
  with outer_zip_file.open("bank.zip") as inner_zip_file:
    with zipfile.ZipFile(BytesIO(inner_zip_file.read())) as inner_zip:
      with inner_zip.open("bank-full.csv") as csv_file:
        data = pd.read_csv(csv_file, sep=';')

print(data.shape)
print(data.head())

(45211, 17)
   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no  


In [ ]:
print(data.info())   # info of dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB
None


####data preprocessing

In [ ]:
data['y'] = data['y'].map({'yes': 1, 'no': 0})   #converting target to numeric as target variable should be numbers

In [ ]:
data_new = pd.get_dummies(data, drop_first = True)  #one hot encoding to convert categorical variabbles to numeric

In [ ]:
X_data = data_new.drop('y', axis=1)   #dropping y
y_data = data_new['y']   # creating y dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, stratify=y_data)       #test train split

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)            #feature scaling
X_test = scaler.transform(X_test)

####performance metrics

In [ ]:
def performance_metrics(y_test,y_pred,y_prob):
  pref_metrics = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "AUC": roc_auc_score(y_test, y_prob),                                    #function for metrics
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1 Score": f1_score(y_test, y_pred),
    "MCC": matthews_corrcoef(y_test, y_pred)}
  return pref_metrics

####Models

1. logistic regression

In [ ]:
result = {}
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)                         #logistic regression model

LogisticRegression(max_iter=1000)

In [23]:
y_pred = logistic_model.predict(X_test)
y_prob = logistic_model.predict_proba(X_test)[:, 1]
result["Logistic Regression"]  = performance_metrics(y_test,y_pred,y_prob)
print("Logistic Regression Metrics:")
for i, j in result["Logistic Regression"].items():
    print(f"{i}: {j:.4f}")

Logistic Regression Metrics:
Accuracy: 0.9012
AUC: 0.9054
Precision: 0.6440
Recall: 0.3488
F1 Score: 0.4525
MCC: 0.4264


In [25]:
cm = confusion_matrix(y_test, logistic_model.predict(X_test))
print(cm)

[[7781  204]
 [ 689  369]]


2.Decision Tree

In [18]:
decision_model = DecisionTreeClassifier(random_state=42)
decision_model.fit(X_train, y_train)
y_pred = decision_model.predict(X_test)
y_prob = decision_model.predict_proba(X_test)[:, 1]
result["Decision Tree"] = performance_metrics(y_test,y_pred,y_prob)
print("Decision Tree Metrics:")
for i, j in result["Decision Tree"].items():
    print(f"{i}: {j:.4f}")

Decision Tree Metrics:
Accuracy: 0.8777
AUC: 0.7135
Precision: 0.4783
Recall: 0.4991
F1 Score: 0.4884
MCC: 0.4191


In [26]:
cm = confusion_matrix(y_test, decision_model.predict(X_test))
print(cm)

[[7409  576]
 [ 530  528]]


3. KNN

In [19]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
y_prob = knn_model.predict_proba(X_test)[:, 1]
result["KNN"] = performance_metrics(y_test,y_pred,y_prob)
print("KNN Metrics:")
for i, j in result["KNN"].items():
    print(f"{i}: {j:.4f}")

KNN Metrics:
Accuracy: 0.8936
AUC: 0.8084
Precision: 0.5860
Recall: 0.3091
F1 Score: 0.4047
MCC: 0.3742


In [27]:
cm = confusion_matrix(y_test, knn_model.predict(X_test))
print(cm)

[[7754  231]
 [ 731  327]]


4. Naive Bayes

In [20]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)
y_prob = nb_model.predict_proba(X_test)[:, 1]
result["Naive Bayes"] = performance_metrics(y_test,y_pred,y_prob)
print("Naive Bayes Metrics:")
for i, j in result["Naive Bayes"].items():
    print(f"{i}: {j:.4f}")

Naive Bayes Metrics:
Accuracy: 0.8639
AUC: 0.8088
Precision: 0.4282
Recall: 0.4877
F1 Score: 0.4560
MCC: 0.3797


In [28]:
cm = confusion_matrix(y_test, nb_model.predict(X_test))
print(cm)

[[7296  689]
 [ 542  516]]


5. Random Forest

In [21]:
rf_model = RandomForestClassifier(n_estimators=100,random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
y_prob = rf_model.predict_proba(X_test)[:, 1]
result["Random Forest"] = performance_metrics(y_test,y_pred,y_prob)
print("Random Forest Metrics:")
for i, j in result["Random Forest"].items():
    print(f"{i}: {j:.4f}")

Random Forest Metrics:
Accuracy: 0.9045
AUC: 0.9272
Precision: 0.6554
Recall: 0.3866
F1 Score: 0.4863
MCC: 0.4561


In [29]:
cm = confusion_matrix(y_test, rf_model.predict(X_test))
print(cm)

[[7770  215]
 [ 649  409]]


6. XGB Boost

In [22]:
xgb_model = XGBClassifier(eval_metric="logloss",random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
y_prob = xgb_model.predict_proba(X_test)[:, 1]
result["XGB Boost"] = performance_metrics(y_test,y_pred,y_prob)
print("XGB Boost Metrics:")
for i, j in result["XGB Boost"].items():
    print(f"{i}: {j:.4f}")

XGB Boost Metrics:
Accuracy: 0.9080
AUC: 0.9291
Precision: 0.6348
Recall: 0.5028
F1 Score: 0.5612
MCC: 0.5149


In [30]:
cm = confusion_matrix(y_test, xgb_model.predict(X_test))
print(cm)

[[7679  306]
 [ 526  532]]


Observations and Conclusion

In [24]:
result_table = pd.DataFrame(result).T
result_table = result_table.round(4)
print(result_table)

                     Accuracy     AUC  Precision  Recall  F1 Score     MCC
Logistic Regression    0.9012  0.9054     0.6440  0.3488    0.4525  0.4264
Decision Tree          0.8777  0.7135     0.4783  0.4991    0.4884  0.4191
KNN                    0.8936  0.8084     0.5860  0.3091    0.4047  0.3742
Naive Bayes            0.8639  0.8088     0.4282  0.4877    0.4560  0.3797
Random Forest          0.9045  0.9272     0.6554  0.3866    0.4863  0.4561
XGB Boost              0.9080  0.9291     0.6348  0.5028    0.5612  0.5149


In [ ]:
observation_table = {
    "Logistic Regression": "It has strong baseline and the accuracy and auc is high but recall value is low.This means it works well but misses positive cases",
    "Decision Tree": "Lower auc and moderate F1 shows it may be overfitting.",
    "KNN": "Accuracy is high but F1 is low , it may be sensitive to scaling.",
    "Naive Bayes": "It has balanced precision and recall, average performance can be due to independence assumptions between features.",
    "Random Forest": "Strong and stable performance with high accuracy and auc.",
    "XGBoost": "Best performance with highest accuracy and auc."
}

observation = pd.DataFrame.from_dict(observation_table, orient='index', columns=["Observation"])
observation


,Observation
Logistic Regression,It has strong baseline and the accuracy and au...
Decision Tree,Lower auc and moderate F1 shows it may be over...
KNN,"Accuracy is high but F1 is low , it may be sen..."
Naive Bayes,"It has balanced precision and recall, average ..."
Random Forest,Strong and stable performance with high accura...
XGBoost,Best performance with highest accuracy and auc.


In [ ]:
print("conclusion : As the dataset is highly imbalanced, most no with some yes and has many categorical variables and some non-linear patterns, therefore XGBoost performed the best as it can handle non-linear and patterns and data imbalance best , along with random forest which is second best in performance, logistic regression is weaker because it is a linear model , and knn is effected due to scaling , decision tree is prone to overfitting")

conclusion : As the dataset is highly imbalanced, most no with some yes and has many categorical variables and some non-linear patterns, therefore XGBoost performed the best as it can handle non-linear and patterns and data imbalance best , along with random forest which is second best in performance, logistic regression is weaker because it is a linear model , and knn is effected due to scaling , decision tree is prone to overfitting
